In [1]:
import pandas as pd

import prepare as prp

import matplotlib.pyplot as plt
import seaborn as sns

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nickolaspedrimiranda/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/nickolaspedrimiranda/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [6]:
repo = pd.read_csv('all_repos.csv')

In [7]:
repo

,repo,language,readme_contents
0,ScottfreeLLC/AlphaPy,Python,AlphaPy\n=======\n\n|badge_pypi| |badge_downlo...
1,jalapic/engsoccerdata,R,"Latest GitHub version: 11/4/2022, v0.1.7\n\nN..."
2,bttmly/nba,JavaScript,# nba\n*Node.js client for nba.com API endpoin...
3,kyleskom/NBA-Machine-Learning-Sports-Betting,Python,# NBA Sports Betting Using Machine Learning 🏀\...
4,linouk23/NBA-Player-Movements,Python,# NBA Player Movements\n\nThis is a script for...
...,...,...,...
792,abanoubragheb1/EventProject,C#,# EventProject\nA project that helps those int...
793,hemvall/Heracles,Vue,# Heracles\nSports Performance Tracker: A REST...
794,Chew/MLB-GameFeed-Bot,Java,# MLB-GameFeed-Bot\n\nAdd to your server with ...
795,paucls/court-booking-ddd-kotlin,Kotlin,# court-booking-ddd\nA sample application usin...


In [4]:
repo = prp.cleanse(repo, 'readme_contents')

In [10]:
valid_languages = ["Python", 'HTML', "JavaScript", "R", "Java", "TypeScript", "PHP", "Ruby", "C#", "C++", "Dart", "Kotlin", "Objective-C", "Swift", "Go", "Rust", "C", "Elixir", "CoffeeScript", "MATLAB", "Visual Basic .NET", "Scala", "Haskell", "Stata", "Haxe", "Lua", "Perl", "Clojure"]

In [ ]:
len(repo)

In [20]:
repo.language.value_counts()

Python               232
JavaScript           141
R                     43
TypeScript            40
HTML                  40
Java                  39
PHP                   35
Ruby                  30
C#                    24
C++                   22
Dart                  22
Kotlin                20
CSS                   19
Vue                   10
Objective-C           10
Swift                 10
Go                     9
Rust                   6
XSLT                   4
TeX                    3
Shell                  3
Solidity               3
C                      3
Elixir                 3
CoffeeScript           2
MATLAB                 2
Visual Basic .NET      2
Svelte                 2
EJS                    2
Matlab                 2
Scala                  2
Haskell                1
Stata                  1
Visual Basic           1
Haxe                   1
Dockerfile             1
XML                    1
Max                    1
Batchfile              1
q                      1


In [9]:
langauges = repo.language.value_counts().index

In [15]:
not_lang = list(set(langauges).difference(set(valid_languages)))

In [21]:
not_lang

['Dockerfile',
 'Max',
 'EJS',
 'Batchfile',
 'Shell',
 'Solidity',
 'XML',
 'XSLT',
 'Visual Basic',
 'q',
 'Vue',
 'Matlab',
 'TeX',
 'CSS',
 'Svelte']